<a href="https://colab.research.google.com/github/Narusi/blog-posts/blob/master/All_seasons_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
assets = ['TLT','SPY','IEF','GLD','DBC']
weights = [0.40,0.3,0.15,0.075,0.075]

prices = yf.download(assets)['Adj Close'].resample('D').fillna(method='ffill').dropna()
ret = prices.pct_change().fillna(0.0)

[*********************100%***********************]  5 of 5 completed


In [4]:
print(ret.describe())
print('\n=================================================================\n')
print(logret.describe())

               DBC          GLD          IEF          SPY          TLT
count  5367.000000  5367.000000  5367.000000  5367.000000  5367.000000
mean     -0.000043     0.000261     0.000149     0.000299     0.000223
std       0.010129     0.009716     0.003453     0.010560     0.007605
min      -0.077844    -0.087808    -0.025073    -0.109424    -0.066683
25%      -0.003268    -0.002402    -0.000964    -0.001277    -0.002180
50%       0.000000     0.000000     0.000000     0.000000     0.000000
75%       0.003667     0.003372     0.001434     0.003056     0.003016
max       0.068744     0.112905     0.034262     0.145198     0.075195


               DBC          GLD          IEF          SPY          TLT
count  5367.000000  5367.000000  5367.000000  5367.000000  5367.000000
mean     -0.000095     0.000213     0.000143     0.000243     0.000194
std       0.010153     0.009727     0.003452     0.010572     0.007601
min      -0.081041    -0.091905    -0.025392    -0.115886    -0.069010
25% 

In [5]:
rebal_index = prices.resample('A').last().index

In [6]:
portfolio = pd.Series(np.zeros(len(rebal_index)), index=rebal_index)
wgt_indices = pd.DataFrame(columns=assets, index=prices.index)

for ai in range(len(assets)):
    wgt_indices[assets[ai]] = weights[ai]

In [7]:
wgts_DF = pd.DataFrame(columns=assets, index=prices.index, dtype=float)
rebCount = 0

for i in wgt_indices.index[1:]:
    for a in assets:
        if i in rebal_index:
            wgt_indices.loc[i,a] = wgt_indices[a][0]
            rebCount += 1
        else:
            wgt_indices.loc[i,a] = wgt_indices.shift(1).loc[i,a] * (1 + ret.loc[i,a])

    wgts_DF.loc[i] = wgt_indices.loc[i].div(wgt_indices.loc[i].sum())

print(f'Rebalancing was done {int(rebCount/5)} times out of {len(rebal_index)}')
wgts_DF.describe()

Rebalancing was done 14 times out of 15


,TLT,SPY,IEF,GLD,DBC
count,5366.000000,5366.000000,5366.000000,5366.000000,5366.000000
mean,0.398333,0.303379,0.148882,0.076702,0.072705
std,0.027815,0.028340,0.005671,0.006413,0.008679
min,0.305433,0.173224,0.136310,0.052989,0.046126
25%,0.385304,0.289012,0.145505,0.073816,0.068382
50%,0.395971,0.306054,0.148012,0.076602,0.072838
75%,0.409842,0.316953,0.151185,0.080222,0.077605
max,0.529396,0.392548,0.184079,0.099373,0.108711


In [8]:
annAssetRet = (prices.iloc[-1]/prices.iloc[0]) ** (365/len(prices.index)) - 1
print(f'Portfolio expected return: {100*round(np.dot(wgts_DF.mean(),annAssetRet),3)}%')

ret.describe()

Portfolio expected return: 3.1%


,DBC,GLD,IEF,SPY,TLT
count,5367.000000,5367.000000,5367.000000,5367.000000,5367.000000
mean,-0.000043,0.000261,0.000149,0.000299,0.000223
std,0.010129,0.009716,0.003453,0.010560,0.007605
min,-0.077844,-0.087808,-0.025073,-0.109424,-0.066683
25%,-0.003268,-0.002402,-0.000964,-0.001277,-0.002180
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.003667,0.003372,0.001434,0.003056,0.003016
max,0.068744,0.112905,0.034262,0.145198,0.075195


In [18]:
portRet = ret.mul(wgts_DF).sum(axis=1)
portInd = pd.Series(np.ones(len(portRet.index))*100, index=portRet.index)
portInd = portInd.mul(portRet.add(1).cumprod())

ann_ret = (portInd[-1]/portInd[0])**(365/len(portRet))-1
print(f'All seasons annualized return ir {100*round(ann_ret,3)}%')

px.line(y=portInd, x=portInd.index).show()

All seasons annualized return ir 10.0%


In [10]:
portRet.describe()

count    5367.000000
mean        0.000270
std         0.003862
min        -0.043674
25%        -0.000815
50%         0.000000
75%         0.001899
max         0.031767
dtype: float64

In [11]:
maxDD = (portInd/portInd.rolling(365).max()-1).min()
print(f'All seasons portfolios maximum drawdown has been {100*round(maxDD,3)}%')

All seasons portfolios maximum drawdown has been -12.6%
